In [1]:
import flet as ft
import numpy as np
import sys
import os
import nest_asyncio
nest_asyncio.apply()

In [2]:
def read_matrix(s): #Verify that user input is in the correct format then convert it to a stochastic numpy array
    mat = [[]]
    row = 0
    if len(s)==0:
        raise AttributeError('Error: Please enter a matrix')
    else:
        if s[0] != '[' or s[-1] !=']':
            raise TypeError('Error: Please enter a matrix in the form [a b c ...;...]')
        else:
            n = 0
            for i in range(1,len(s)): #Read character, check what it is and act accordingly
                v = s[i]
                if v not in ['0','1','2','3','4','5','6','7','8','9',' ',';',']']:
                    raise TypeError('Error: The only accepted symbols are 0-9, ; , blankspace, [ and ]')
                else:
                    if v == ';':
                        mat[row].append(n)
                        n = 0
                        mat.append([]) #New line
                        row += 1
                    elif v == ' ' or v ==']':
                        mat[row].append(n)
                        n = 0
                    else:
                        n = 10*n+int(v)

    convposs=True #Is numpy.array() call possible
    lens = [len(i) for i in mat]
    l0 = len(lens)
    for i in range(l0):
        if lens[i]!=l0:
            convposs=False
            break
    
    if not convposs:
        raise ValueError("Error: Matrix is rectangular or its lines don't all have the same length, beware the number of spaces")
    else:
        mat = np.array(mat)
        sum = np.sum(mat,axis=1)[:,None]
        if not sum.all():
            raise ValueError('Error: Matrix P is not stochastic')
        else:
            mat= mat/sum
            eigvals = np.linalg.eigvals(mat)
            if (np.abs(np.abs(eigvals)-1)<1e-6).sum()>1: #If the matrix has an eigenvalue different from 1 whose absolute value(or complex modulus) is equal to 1 then the markov chain is aperiodic, if the eigenvalue 1 has multiplicity greater than 1, the markov chain is not irreductible
                raise ValueError('Error: The Markov chain induced by your matrix is either periodic or reductible, verify that the induced graph is strongly connex and aperiodic')
            else:
                return mat
        

def p_inf(m): #Computes p_infinity
    mat = m.copy()
    n = len(mat)
    I = np.eye(n)
    IP = I-mat
    IPT = IP.T
    IPT[0,:] = np.ones(n)
    v = np.eye(1,n,0).reshape(-1)
    p = np.linalg.inv(IPT)@v
    return p





def main(page: ft.Page):

    page.title = 'Exercise 2'
    page.window.width = 700
    page.window.height = 500
    
 
    page.add(ft.Text('Calculation of p_infinity from P',width=700,height=50,text_align=ft.TextAlign.CENTER,weight=ft.FontWeight.BOLD,size=20))

    matrix = ft.TextField('[1 3;2 8]',label='Enter matrix, ex: [1 2;3 4]')
    page.add(ft.Row(controls=[ft.Text('Probability Matrix P (to be normalized)'),matrix],height=75))#Row containing the text field to enter the matrix

    mattext = ft.Text()
    matdisplay = ft.Text()
    result = ft.Text()
    page.add(ft.Row([mattext,matdisplay,result],height=100))#Row that will display the results


    def exit(e):
        page.window.close()

    def compute(e):#Called when you click on button 'COMPUTE', computes P and p_infinity and displays them, displays error message if one occurs
        val = matrix.value
        try:
            val = read_matrix(val)
            p = p_inf(val)
            mattext.value = 'P ='
            matdisplay.value = str(val)
            result.value = '; p_infinity ='+str(p)
            errors.value = ""
        except Exception as ex:
            #exc_type, exc_obj, exc_tb = sys.exc_info()
            #fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            #print(exc_type, fname, exc_tb.tb_lineno)
            errors.value = ex
        page.update()
        
    page.add(ft.Row([ft.ElevatedButton(text = 'EXIT',on_click=exit),ft.ElevatedButton(text='COMPUTE',on_click=compute)],alignment=ft.MainAxisAlignment.SPACE_EVENLY))#Row containing the buttons to use the app

    errors = ft.Text()#Error message area
    page.add(errors)


    page.update()
    compute(0)


In [ ]:
# Run the app
# Works only on Windows 11 and macOS. If using Windows 10, comment this line and use the web browser option below.
ft.app(target=main,view=ft.AppView.FLET_APP)

In [ ]:
# Works on all operating systems, including Windows 10. Uncomment this line if using Windows 10 and comment the pop-up window option.
#ft.app(target=main,view=ft.AppView.WEB_BROWSER)